# 0. Data selection and necessary package importing¶
We select the SPDR Gold Shares (GLD) ETF as the gold ETF.

Similarly, we choose the SPDR Portfolio Europe ETF (SPEU) as the equity ETF.

The currency for Bitcoin is USD.

In [1]:
import arch
import holidays
import pmdarima
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import VECM
from sklearn import preprocessing
%matplotlib inline

# 1. Data importing


In [2]:
btc_df=pd.read_csv("btc 2020.csv",index_col="Date",parse_dates=True)
gold_df=pd.read_csv('gold 2020.csv',index_col="Date",parse_dates=True)
eqty_df=pd.read_csv('eqty 2020.csv',index_col="Date",parse_dates=True)
del eqty_df['T/F']

In [3]:
btc_df.shape

(361, 6)

In [4]:
gold_df.shape

(252, 6)

In [5]:
eqty_df.shape

(252, 6)

We have to ensure that all three datasets have data for same and equal number of days to make cointegration operation in part 7 apply properly.

In [6]:
missing = set(btc_df.index).symmetric_difference(set(gold_df.index))
def remove_missing_days(df, indices, missing):
    return df.loc[[index for index in indices if index not in missing]]

In [7]:
gold_df = remove_missing_days(gold_df, eqty_df.index, missing)
eqty_df = remove_missing_days(eqty_df, eqty_df.index, missing)
btc_df = remove_missing_days(btc_df, eqty_df.index, missing)


In [8]:
#to confirm
assert eqty_df.shape == gold_df.shape
assert eqty_df.shape == btc_df.shape

# 2. Data Processing 
We use adjusted close prices to calculate the daily returns. Adjusted close prices are the prices that already take into account stock split and dividends, which reflex more accurate the change of the prices

In [9]:
gold_df['Daily_return_pct'] = gold_df['Adj Close'].pct_change(1)
gold_df.loc["2020-04-01":].head()

Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-04-01  148.199997  150.080002  147.850006  149.449997  149.449997   
2020-04-02  151.199997  152.500000  150.699997  151.899994  151.899994   
2020-04-03  152.229996  153.089996  151.660004  152.649994  152.649994   
2020-04-06  154.160004  157.220001  153.960007  156.880005  156.880005   
2020-04-07  155.330002  156.139999  154.380005  156.039993  156.039993   

              Volume  Daily_return_pct  
Date                                    
2020-04-01  11827400          0.009456  
2020-04-02   9188300          0.016393  
2020-04-03   8469100          0.004937  
2020-04-06  14739300          0.027711  
2020-04-07   9195400         -0.005354

In [10]:
eqty_df['Daily_return_pct'] = eqty_df['Adj Close'].pct_change(1)
eqty_df.loc['2020-04-01':].head()

Open       High        Low      Close  Adj Close  Volume  \
Date                                                                        
2020-04-01  26.020000  26.320000  25.709999  25.809999  24.906504   15600   
2020-04-02  25.980000  26.280001  25.820000  26.209999  25.292501   29800   
2020-04-03  25.900000  26.120001  25.530001  25.660000  24.761751   26400   
2020-04-06  26.530001  27.049999  26.530001  27.049999  26.103096   23100   
2020-04-07  27.840000  27.840000  27.150000  27.200001  26.247845   50700   

            Daily_return_pct  
Date                          
2020-04-01         -0.041945  
2020-04-02          0.015498  
2020-04-03         -0.020984  
2020-04-06          0.054170  
2020-04-07          0.005545

In [11]:
btc_df['Daily_return_pct'] = btc_df['Adj Close'].pct_change(1)
btc_df.loc['2020-04-01':].head()

Open         High          Low        Close    Adj Close  \
Date                                                                          
2020-04-01  6437.319336  6612.573730  6202.373535  6606.776367  6606.776367   
2020-04-02  6606.776367  7088.247559  6595.918457  6793.624512  6793.624512   
2020-04-03  6797.396484  7003.220703  6673.335938  6733.387207  6733.387207   
2020-04-06  6788.049805  7271.781250  6782.889648  7271.781250  7271.781250   
2020-04-07  7273.644043  7427.939453  7136.714355  7176.414551  7176.414551   

                 Volume  Daily_return_pct  
Date                                       
2020-04-01  40346426266          0.026113  
2020-04-02  47660646124          0.028281  
2020-04-03  38976504903         -0.008867  
2020-04-06  46896904615          0.079959  
2020-04-07  44243482668         -0.013115

# 5. Category 1 Models: Just use 1 variable.

In [12]:
df_names = {0: "gold ETF", 1: "equity ETF", 2: "Bitcoin"}
dfs = [gold_df, eqty_df, btc_df]
def get_data(df, month_start, month_end, column=None):
    data = df[(df.index >= f"2020-{month_start:02d}-01") & (df.index < f"2020-{month_end:02d}-01")]
    if column: 
        data = data[column]
    return data

In [13]:
def summarize_data(df):
    data = get_data(df, 3, 12, "Adj Close")
    moving_avg = data.rolling(20, min_periods=1).mean()
    return get_data(moving_avg, 4, 12)


In [14]:
def get_data(df, month_start, month_end, column):
    return df[(df.index >= f"2020-{month_start:02d}-01") & (df.index < f"2020-{month_end:02d}-01")][column]

In [15]:
def fit_arima(data, exog= None):
    model = ARIMA(data, exog=exog, order=(2,0,2))
    model_fit = model.fit()
    return model_fit

In [16]:
def fit_garch(data, garch_type="GARCH"):
    if garch_type == "TARCH":
        garch = arch.arch_model(data, vol='TGARCH', p=1, o=1, q=1, power=1)
    else:
        garch = arch.arch_model(data, vol=garch_type, p=1, o=0, q=1)
    garch_fitted = garch.fit()
    print(garch_fitted.summary())
    if garch_type != "FIGARCH":
        omega = garch_fitted.params["omega"]
        alpha = garch_fitted.params["alpha[1]"]
        beta = garch_fitted.params["beta[1]"]
        print(f"Unconditional variance: {omega/(1 - alpha - beta)}")
    return garch_fitted

In [17]:
btc_q2 = get_data(btc_df, 4, 6, "Daily_return_pct")
btc_q3 = get_data(btc_df, 7, 9, "Daily_return_pct")
btc_q4 = get_data(btc_df, 10, 12, "Daily_return_pct")

In [18]:
eqty_q2 = get_data(eqty_df, 4, 6, "Daily_return_pct")
eqty_q3 = get_data(eqty_df, 7, 9, "Daily_return_pct")
eqty_q4 = get_data(eqty_df, 10, 12, "Daily_return_pct")

In [19]:
gold_q2 = get_data(gold_df, 4, 6, "Daily_return_pct")
gold_q3 = get_data(gold_df, 7, 9, "Daily_return_pct")
gold_q4 = get_data(gold_df, 10, 12, "Daily_return_pct")

In [20]:
btc_q2.shape

(40,)

In [21]:
btc_q3.shape

(43,)

In [22]:
btc_q4.shape

(39,)

In [23]:
gold_q2.shape

(40,)

In [24]:
gold_q3.shape

(43,)

In [25]:
gold_q4.shape

(39,)

Our first model with ARIMA is a simple one, we'll allocate our capital totally to bitcoin if we predict that the next day price will rise, otherwise we will short 100%.

The total return for Q3 is 27.83%, not very impressive, but for Q4, it's 81.92%, which is much better. Return for 2 quarters is 132.55%

In [26]:
model = fit_arima(btc_q2)
return_pred = model.predict(start=btc_q2.shape[0], end=btc_q2.shape[0] + btc_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(btc_q3 * signal + 1) - 1
return_q3

C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\anaconda\lib\site-packages

0.2782697579614861

In [27]:
model = fit_arima(btc_q3)
return_pred = model.predict(start=btc_q3.shape[0], end=btc_q3.shape[0] + btc_q4.shape[0] - 1).values
signal = np.where(return_pred >0, 1, -1)
return_q4 = np.product(btc_q4 * signal + 1) - 1
return_q4

C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


0.8192927900221496

In [28]:
(1 + return_q3) * (1 + return_q4) - 1

1.3255469543626899

ARMA model is preferrable for its ease of application leading to a decent return decent return

# 6. Category 2 Models: Just use 2 variables
With VARMA model, we use the gold returns as to enhance the prediction for the bitcoin returns. for Q3, the return is 42.5% while for Q4, it's 40.8%. Return for the 2 quarters is 100.7%.

In [29]:
model = fit_arima(btc_q2, gold_q2)
return_pred = model.predict(exog=gold_q3, start=btc_q2.shape[0], end=btc_q2.shape[0] + btc_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(btc_q3 * signal + 1) - 1
return_q3

C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\anaconda\lib\site-packages

0.4254301818289916

In [30]:
model = fit_arima(btc_q3, gold_q3)
return_pred = model.predict(exog=gold_q4, start=btc_q3.shape[0], end=btc_q3.shape[0] + btc_q4.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q4 = np.product(btc_q4 * signal + 1) - 1
return_q4

C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\anaconda\lib\site-packages

0.4080149519257539

In [31]:
(1 + return_q3) * (1 + return_q4) - 1

1.0070270089414661

We prefer the VARMA model thanks to its execellent return

# 7 . Category 3 Models: Use all 3 variables


## VARMA model
With VARMA model, we use both the equity and gold returns as to enhance the prediction for the bitcoin returns. for Q3, the return is 56.6% while for Q4, it's 61.59%. Return for 2 quarters is 153%

In [32]:
model = fit_arima(btc_q2, pd.concat([eqty_q2, gold_q2], axis=1))
return_pred = model.predict(exog=pd.concat([eqty_q3, gold_q3], axis=1), start=btc_q2.shape[0], end=btc_q2.shape[0] + btc_q3.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q3 = np.product(btc_q3 * signal + 1) - 1
return_q3

C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\anaconda\lib\site-packages

0.5660346210556637

In [33]:
model = fit_arima(btc_q3, pd.concat([eqty_q3, gold_q3], axis=1))
return_pred = model.predict(exog=pd.concat([eqty_q4, gold_q4], axis=1), start=btc_q3.shape[0], end=btc_q3.shape[0] + btc_q4.shape[0] - 1).values
signal = np.where(return_pred > 0, 1, -1)
return_q4 = np.product(btc_q4 * signal + 1) - 1
return_q4

C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:218: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
C:\anaconda\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressi

0.6159417075655702

In [34]:
(1 + return_q3) * (1 + return_q4) - 1

1.53062065965549

Compare models using cumulative return and volatility

In [35]:
data1 = get_data(gold_df, 4, 6, "Daily_return_pct").values
data2 = get_data(eqty_df, 4, 6, "Daily_return_pct").values
data3 = get_data(btc_df, 4, 6, "Daily_return_pct").values

In [36]:
coint(data1, np.array([data2, data3]).T)

(-5.498262846629299,
 9.134843608382127e-05,
 array([-4.68490329, -3.96689229, -3.61396547]))

In [37]:
coint(data2, np.array([data1, data3]).T)

(-7.789803152502089,
 3.151045891763875e-10,
 array([-4.68490329, -3.96689229, -3.61396547]))

In [38]:
coint(data3, np.array([data1, data2]).T)

(-5.806180166821672,
 2.165043729264147e-05,
 array([-4.68490329, -3.96689229, -3.61396547]))

p-value < 0.01 for all tests, we conclude that there are cointegrating vectors.

## Vector Error Correction Model

We allocate our capital to 3 assets based on the predicted return, with a negative return means that we'll short the asset, and the sum of the 3 assets is always 1. Q3's return is 7%, Q4's return is 46.3%, total return is 56.5%

In [39]:
data_q2 = np.array([data1, data2, data3]).T
model = VECM(data_q2, coint_rank=1)
vecm_res = model.fit()
res = vecm_res.predict(steps=43)
row_sums = res.sum(axis=1)
weights = res / row_sums[:, np.newaxis]
weights

array([[ 0.262222  ,  0.88870077, -0.15092277],
       [ 0.82906013,  1.07910192, -0.90816205],
       [ 0.37093187,  0.63235958, -0.00329145],
       [ 0.57971912,  0.89941556, -0.47913467],
       [ 0.42946727,  0.72361578, -0.15308305],
       [ 0.51204905,  0.81523669, -0.32728574],
       [ 0.45322013,  0.7463966 , -0.19961673],
       [ 0.49032743,  0.79122904, -0.28155646],
       [ 0.46525192,  0.76117518, -0.2264271 ],
       [ 0.48132628,  0.78027728, -0.26160356],
       [ 0.47058059,  0.76747389, -0.23805448],
       [ 0.47760186,  0.77586896, -0.25347082],
       [ 0.4729494 ,  0.77030697, -0.24325637],
       [ 0.47600014,  0.77395052, -0.24995066],
       [ 0.47398516,  0.77154406, -0.24552921],
       [ 0.47531036,  0.77312725, -0.24843761],
       [ 0.47443636,  0.77208304, -0.2465194 ],
       [ 0.47501164,  0.77277029, -0.24778193],
       [ 0.47463249,  0.77231736, -0.24694985],
       [ 0.47488217,  0.77261564, -0.24749781],
       [ 0.47471766,  0.7724191 , -0.247

In [40]:
data1 = get_data(gold_df, 7, 9, "Daily_return_pct").values
data2 = get_data(eqty_df, 7, 9, "Daily_return_pct").values
data3 = get_data(btc_df, 7, 9, "Daily_return_pct").values
data_q3 = np.array([data1, data2, data3]).T

In [41]:
return_q3 = weights * data_q3
total_return_q3 = return_q3.sum(axis=1)
portfolio_return_q3 = np.product(total_return_q3 + 1) - 1
portfolio_return_q3

0.06973922213354022

In [42]:
coint(data1, np.array([data2, data3]).T)

(-6.727406000681917,
 1.768061809888097e-07,
 array([-4.6556178 , -3.95031996, -3.60224723]))

In [43]:
coint(data2, np.array([data1, data3]).T)

(-5.409426190439604,
 0.00013605598341834754,
 array([-4.6556178 , -3.95031996, -3.60224723]))

In [44]:
coint(data3, np.array([data1, data2]).T)

(-7.785850214705772,
 3.230366391094859e-10,
 array([-4.6556178 , -3.95031996, -3.60224723]))

 We can reject the null hypothesis of no cointegration for all three cases above as p value < 0.01.

In [45]:
model = VECM(data_q3, coint_rank=1)
vecm_res = model.fit()
res = vecm_res.predict(steps=39)
row_sums = res.sum(axis=1)
weights = res / row_sums[:, np.newaxis]
weights

array([[0.25743814, 0.06174887, 0.68081299],
       [0.34163356, 0.07092187, 0.58744457],
       [0.25978739, 0.10076549, 0.63944712],
       [0.31445324, 0.0420693 , 0.64347747],
       [0.26595903, 0.08444191, 0.64959907],
       [0.30147175, 0.06350738, 0.63502086],
       [0.2761895 , 0.07888215, 0.64492835],
       [0.29232878, 0.06739783, 0.64027339],
       [0.2816594 , 0.07481268, 0.64352792],
       [0.28842299, 0.07060677, 0.64097024],
       [0.28427145, 0.07312501, 0.64260354],
       [0.28670833, 0.07158546, 0.64170621],
       [0.28530719, 0.07246371, 0.6422291 ],
       [0.28608689, 0.07200563, 0.64190748],
       [0.28567482, 0.07223566, 0.64208951],
       [0.28587748, 0.07212487, 0.64199765],
       [0.28578755, 0.07217057, 0.64204188],
       [0.28582   , 0.07215861, 0.64202139],
       [0.28581496, 0.07215592, 0.64202912],
       [0.28580861, 0.07216345, 0.64202795],
       [0.28581826, 0.07215527, 0.64202647],
       [0.28580894, 0.07216244, 0.64202862],
       [0.

In [46]:
data1 = get_data(gold_df, 10, 12, "Daily_return_pct").values
data2 = get_data(eqty_df, 10, 12, "Daily_return_pct").values
data3 = get_data(btc_df, 10, 12, "Daily_return_pct").values
data_q4 = np.array([data1, data2, data3]).T

In [47]:
return_q4 = weights * data_q4
total_return_q4 = return_q4.sum(axis=1)
portfolio_return_q4 = np.product(total_return_q4 + 1) - 1
portfolio_return_q4

0.4633518421420779

In [48]:
(1 + portfolio_return_q3) * (1 + portfolio_return_q4) - 1

0.5654048613207496

We prefer the VARMA model because it has a better return

# 8. Combining the models

In [90]:
#????? pls do this part, i'm a little confused abt it